In [11]:
import re
import csv
import math
import spacy
import pandas as pd
from spacy.symbols import nsubj, nsubjpass, dobj, iobj, pobj
from collections import Counter, defaultdict
from itertools import islice
from html2text import html2text
from fog.tokenizers import ngrams
from tqdm import tqdm_notebook

In [2]:
nlp = spacy.load('fr')

In [38]:
DATA = '../amds_vectors.csv'
STOPWORDS = './stopwords.txt'
BIGRAMS = './bigrams.csv'
LABELS = set([nsubj, nsubjpass, dobj, iobj, pobj])

In [7]:
WHITESPACE_RE = re.compile(r'\n+')
AMENDEMENTS = []
with open(DATA, 'r') as f:
    reader = csv.DictReader(f)
    
    for line in reader:
        line['expose'] = WHITESPACE_RE.sub(' ', html2text(line['expose']))
        AMENDEMENTS.append(line)

In [14]:
STOPWORDS_SET = set()

with open(STOPWORDS, 'r') as f:
    for line in f.readlines():
        line = line.strip()
        
        if line:
            STOPWORDS_SET.add(line)

In [46]:
BIGRAMS_SET = {}

with open(BIGRAMS, 'r') as f:
    reader = csv.DictReader(f)
    
    for line in reader:
        if line['Class'] == 'drop' or not line['Class']:
            continue
        BIGRAMS_SET[(line['A'], line['B'])] = line['Class']

In [47]:
set(category for category in BIGRAMS_SET.values())

{'actor', 'institution', 'intention', 'support', 'thel', 'theme', 'type'}

In [10]:
pd.DataFrame(AMENDEMENTS).head(100)

,expose,nb_signataires,groupes,expose_ln,duplicates,signataires,sort,concurrents,text_ln,legislature,id,signataire_special,texte,id_dossier_an,auteur_groupe_acronyme,majorite_opposition
0,Le présent amendement demande au gouvernement ...,2,LREM,587,1,"M. Sommer, M. Barbier",Retiré,8,250,15,32772,,"<p>Le Gouvernement, après consultation des org...",choix_avenir_professionnel,LREM,majorité
1,Cet amendement vise à simplifier les obligatio...,14,LR,1132,1,"M. Hetzel, M. Marleix, M. Straumann, M. Quent...",Retiré,1,1381,15,5,,<p>Le chapitre II du titre XI du livre II de l...,habilitation_ordonnances_dialogue_social,LR,opposition
2,Les centres de formation d'apprentis jouent un...,15,LREM,644,1,"Mme Fontaine-Domeizel, M. Taquet, Mme Bagarry...",Retiré,33,484,15,32776,,"<p>Après l'alinéa 25, insérer l'alinéa suivant...",choix_avenir_professionnel,LREM,majorité
3,"Les partenaires sociaux avaient déjà lancé, lo...",1,LR,1720,1,M. Hetzel,Non soutenu,3,2207,15,12,,<p>Après l'article L. 1242-2 du code du travai...,habilitation_ordonnances_dialogue_social,LR,opposition
4,La formulation exposée dans le présent texte «...,3,LR,229,1,"M. Viala, M. Cinieri, M. Straumann",Non soutenu,7,149,15,15,,"<p>À l'alinéa 7, après le mot :</p><p>« seuil ...",habilitation_ordonnances_dialogue_social,LR,opposition
5,Cet amendement vise à combler une lacune du pr...,17,LR,169,2,"M. Descoeur, M. Sermier, M. Bony, M. Leclerc,...",Non soutenu,89,291,15,32784,,<p>Compléter l'alinéa 8 par la phrase suivante...,choix_avenir_professionnel,LR,opposition
6,La loi du 17 août 2015 n'a pas gagné le pari d...,14,LR,264,1,"M. Hetzel, M. Marleix, M. Straumann, M. Quent...",Retiré,2,708,15,18,,<p>I. – L'article 1er de la loi n° 2015-994 du...,habilitation_ordonnances_dialogue_social,LR,opposition
7,La loi n° 2016‑1657 du 5 décembre 2016 relativ...,14,"GDR,LR,LREM,MODEM,NG,UAI",4124,1,"Mme Bareigts, Mme Benin, Mme Manin, M. Serva,...",Adopté,23,977,15,32787,,"<p>I. – Après l'alinéa 18, insérer l'alinéa su...",choix_avenir_professionnel,NG,mixte
8,,14,LR,0,1,"M. Cherpion, M. Viry, M. Jacob, Mme Brenier, ...",Irrecevable,1,74,15,32788,,<p>Amendement irrecevable au titre de l'articl...,choix_avenir_professionnel,LR,opposition
9,Afin de garantir le suivi des candidatures et ...,3,LREM,484,1,"Mme Piron, Mme Lazaar, M. Pellois",Retiré avant séance,9,230,15,32793,,"<p>Après l'alinéa 1, insérer l'alinéa suivant ...",choix_avenir_professionnel,LREM,majorité


In [28]:
BIGRAMS_PER_GROUPE = defaultdict(Counter)
DF = defaultdict(set)
GROUPES = set()
BIGRAMS = set()
for amd in tqdm_notebook(AMENDEMENTS):
    doc = nlp(amd['expose'])

    for sentence in doc.sents:
        tokens = [token.text.lower() for token in sentence if token.is_alpha and token.text.lower() not in STOPWORDS_SET]
        
        for gram in ngrams(2, tokens):
            for groupe in amd['groupes'].split(','):
                GROUPES.add(groupe)
                gram = tuple(gram)
                BIGRAMS_PER_GROUPE[groupe][gram] += 1
                DF[gram].add(groupe)
                BIGRAMS.add(gram)
            

In [50]:
N = len(GROUPES)

for groupe in GROUPES:
    print(groupe)
    print(len(groupe) * '-')
    
    tfidfs = Counter()
    
    for bigram, n in BIGRAMS_PER_GROUPE[groupe].items():
        
        if BIGRAMS_SET.get(bigram) != 'theme':
            continue
        
        tfidfs[bigram] = n * math.log(N / len(DF[bigram]))
        
    for bigram, n in tfidfs.most_common(25):
        print(bigram, n)
    
    print()

LR
--
('logements', 'individuels') 54.72088902275941
('haut', 'débit') 45.81453659370776
('espaces', 'publics') 37.32315693410402
('communes', 'nouvelles') 36.7368005696771
('communes', 'intercommunalités') 34.81904781121789
('commun', 'urbanisme') 34.81904781121789
('grandes', 'opérations') 31.671188673491425
('taxes', 'foncières') 31.153884883721275
('regroupement', 'familial') 30.23759415184712
('mutuelles', 'étudiantes') 25.656140492476343
('commune', 'nouvelle') 22.90726829685388
('chefs', 'entreprise') 20.94385057440562
('revenu', 'fiscal') 18.900548079341657
('réseaux', 'mobiles') 18.714973875118524
('zones', 'rurales') 18.389722455575665
('équipements', 'publics') 17.62834055382257
('produits', 'phytopharmaceutiques') 17.62834055382257
('opérations', 'aménagement') 17.477072252997885
('communications', 'électroniques') 16.857245584277695
('qualité', 'air') 16.493233173734794
('réserve', 'parlementaire') 15.693697533304226
('construction', 'habitation') 15.396905040680474
('rela